Things to do: a) get a dataset of coq code files
b) create a baseline model similar to what they did in the paper (n-grams or RNN) 
c) create a transformer model to complete the same task
d) compare the results of both models and save the transformer

In [91]:
import pandas as pd
import numpy as np
import nltk
from nltk.util import ngrams
from collections import defaultdict, Counter

In [92]:
df = pd.read_parquet('props-proofs.parquet').head(150)
df_test = df.iloc[100:150]
df = df.iloc[:100]

In [93]:
df['combined'] = df['proposition'] + ' ' + df['proof']
df.drop(columns=['proposition', 'proof', 'imports', 'filename', 'symbolic_name'], inplace=True)

In [94]:
def find_unique_symbols(df, column_name):
    """
    This function finds all unique symbols occuring in our dataset and changes it to a set. 
    """
    all_text = ''.join(df[column_name].dropna().astype(str))
    unique_symbols = set(all_text)
    
    return unique_symbols


In [95]:
import re

def create_tokenizer(symbol_set):
    """
    This function creates a tokenizer that changes a string into words, symbols, and whitespace tokens.
    An exception is added for variable names so that they are not split into individual characters. i.e. 'var_name' is one token instead of 'var' '_' 'name'.

    """
    filtered_symbols = {s for s in symbol_set if not s.isalpha()}

    symbol_pattern = '|'.join(re.escape(s) for s in sorted(filtered_symbols, key=len, reverse=True))

    def tokenizer(input_string):
        tokens = re.findall(rf'\b\w+(?:_\w+)*\b|{symbol_pattern}|\s|[^\w\s]', input_string)
        return tokens

    return tokenizer


symbol_set = find_unique_symbols(df, 'combined')
tokenizer = create_tokenizer(symbol_set)

In [96]:
data = df.apply(lambda x: tokenizer(x['combined']), axis=1).tolist()

In [97]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from collections import Counter
import torch.optim as optim


# 1. Klasa dataset dla tokenów
class TokenDataset(Dataset):
    """
    This class created the dataset for the model from our given cleaned and tokenized data.
    """
    def __init__(self, data, vocab=None, max_vocab_size=30000):
        self.data = data
        
        if vocab is None:
            token_counts = Counter(token for line in data for token in line)
            most_common_tokens = token_counts.most_common(max_vocab_size - 2)
            self.vocab = {token: idx + 2 for idx, (token, _) in enumerate(most_common_tokens)}
            self.vocab['<PAD>'] = 0
            self.vocab['<UNK>'] = 1
        else:
            self.vocab = vocab

        self.vocab_size = len(self.vocab)
        self.pad_token = self.vocab['<PAD>']

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        tokens = self.data[idx]
        # before the token text
        input_tokens = tokens[:-1] 
        # token to predict
        target_token = tokens[1:] 
        
        input_indices = [self.vocab.get(token, self.vocab['<UNK>']) for token in input_tokens]
        target_indices = [self.vocab.get(token, self.vocab['<UNK>']) for token in target_token]

        return torch.tensor(input_indices, dtype=torch.long), torch.tensor(target_indices, dtype=torch.long)

def collate_fn(batch):
    """
    This function prepares the batches for the model.
    """
    inputs, targets = zip(*batch)
    inputs_padded = nn.utils.rnn.pad_sequence(inputs, batch_first=True, padding_value=0)
    targets_padded = nn.utils.rnn.pad_sequence(targets, batch_first=True, padding_value=0)
    return inputs_padded, targets_padded

class TransformerModel(nn.Module):
    """
    This is our transformer model class
    """
    def __init__(self, vocab_size, embedding_dim=512, nhead=8, num_layers=6, dim_feedforward=2048, dropout=0.1):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.transformer = nn.Transformer(
            d_model=embedding_dim,
            nhead=nhead,
            num_encoder_layers=num_layers,
            num_decoder_layers=num_layers,
            dim_feedforward=dim_feedforward,
            dropout=dropout
        )
        self.fc_out = nn.Linear(embedding_dim, vocab_size)
        
    def forward(self, src, tgt, src_mask=None, tgt_mask=None):
        src_emb = self.embedding(src)  # [seq_length, batch_size, embedding_dim]
        tgt_emb = self.embedding(tgt)
        
        transformer_output = self.transformer(src_emb, tgt_emb, src_mask=src_mask, tgt_mask=tgt_mask)
        output = self.fc_out(transformer_output)  # [seq_length, batch_size, vocab_size]
        
        return output

def generate_square_subsequent_mask(size):
    """
    This function ensures that the model does not look into the future tokens.
    """
    mask = torch.triu(torch.ones(size, size) * float('-inf'), diagonal=1)
    return mask


def train_model(model, dataloader, criterion, optimizer, epochs=5):
    """
    This function trains the given model using the given dataloader, criterion, and optimizer.
    """
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for src, tgt in dataloader:
            src, tgt = src.transpose(0, 1), tgt.transpose(0, 1)
            
            src_mask = generate_square_subsequent_mask(src.size(0))
            tgt_mask = generate_square_subsequent_mask(tgt.size(0) - 1)
            
            output = model(src, tgt[:-1], src_mask=src_mask, tgt_mask=tgt_mask)
            output_dim = output.shape[-1]
            
            output = output.reshape(-1, output_dim)
            tgt = tgt[1:].reshape(-1)
            loss = criterion(output, tgt)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(dataloader)}")


dataset = TokenDataset(data, max_vocab_size=30000)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
embedding_dim = 512
model = TransformerModel(vocab_size=dataset.vocab_size, embedding_dim=embedding_dim)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss(ignore_index=dataset.pad_token)

train_model(model, dataloader, criterion, optimizer, 25)
model_path = 'transformer_model.pth'
torch.save(model.state_dict(), model_path)
print(f'Model saved to {model_path}')

c:\Users\SzymonWoźniak(272471\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch 1/25, Loss: 4.434102118015289
Epoch 2/25, Loss: 3.4872068762779236
Epoch 3/25, Loss: 3.3145312666893005
Epoch 4/25, Loss: 3.314699351787567
Epoch 5/25, Loss: 3.2058945894241333
Epoch 6/25, Loss: 3.0034059882164
Epoch 7/25, Loss: 2.794138729572296
Epoch 8/25, Loss: 2.619040071964264
Epoch 9/25, Loss: 2.527340352535248
Epoch 10/25, Loss: 2.5188753604888916
Epoch 11/25, Loss: 2.3026849031448364
Epoch 12/25, Loss: 2.2505701780319214
Epoch 13/25, Loss: 2.192489266395569
Epoch 14/25, Loss: 2.181410849094391
Epoch 15/25, Loss: 2.2047762870788574
Epoch 16/25, Loss: 2.06540584564209
Epoch 17/25, Loss: 1.98635995388031
Epoch 18/25, Loss: 1.9873154163360596
Epoch 19/25, Loss: 1.8804767429828644
Epoch 20/25, Loss: 1.8210581839084625
Epoch 21/25, Loss: 1.8498771786689758
Epoch 22/25, Loss: 1.8142377138137817
Epoch 23/25, Loss: 1.7478831708431244
Epoch 24/25, Loss: 1.745235115289688
Epoch 25/25, Loss: 1.6406884789466858
Model saved to transformer_model.pth


In [98]:
import torch.nn.functional as F

def predict_next_token(model, sequence, vocab, max_len=50):
    """
    This function predicts the next token based on the given sequence.
    
    Inputs:
    - model: trained transformer model
    - sequence: base sequence before the predictied token
    - vocab: dictionalry mapping tokens to indexes
    - max_len: maximum amount of tokens to generate
    
    Outputs
    - Next given tokens
    """
    model.eval()
    
    input_indices = [vocab.get(token, vocab['<UNK>']) for token in sequence]
    input_tensor = torch.tensor(input_indices, dtype=torch.long).unsqueeze(1)
    src_mask = generate_square_subsequent_mask(input_tensor.size(0))
    
    with torch.no_grad():
        output = model(input_tensor, input_tensor, src_mask=src_mask, tgt_mask=src_mask)
        output = output[-1, 0, :]
        
        probabilities = F.softmax(output, dim=-1)
        
        # most probable next token is chosen
        predicted_index = torch.argmax(probabilities).item()
        # print(predicted_index)
        idx_to_token = {idx: token for token, idx in vocab.items()}
        # print(idx_to_token)
        predicted_token = idx_to_token.get(predicted_index, '<UNK>')
    
    return predicted_token
test_sequence = ['Lemma', ' ', 'scons_comp', ' ', 'x', ' ', 'f', ' ', 'g', ' ', ':', ' ', '(', 'x', ' ', '.', ':', ' ', 'f', ')', ' ', '>', '>', '>', ' ', 'g', ' ', '=', ' ', '(', 'g', ' ', 'x', ')', ' ', '.', ':', ' ', 'f', ' ', '>', '>', '>', ' ', 'g', '.', ' ', 'Proof', '.', ' ', 'f_ext', ';', ' ', 'let', ' ', 'x', ' ', ':', '=', ' ', 'fresh', ' ', 'in', ' ', 'intros', ' ', 'x', ';', ' ', 'now', ' ', 'destruct', ' ', 'x', '.', ' ', 'Qed']


predicted_token = predict_next_token(model, test_sequence, dataset.vocab)
print("Predicted token: '" + predicted_token+"'")

Predicted token: '.'


In [ ]:
tokens = [token for line in data for token in line]
n = 2
bigrams = list(ngrams(tokens, n))

bigram_freq = defaultdict(Counter)
for w1, w2 in bigrams:
    bigram_freq[w1][w2] += 1

def predict_next_token(prev_token):
    if prev_token in bigram_freq:
        return bigram_freq[prev_token].most_common(1)[0][0]
    else:
        return None

prev_token = "Qed"
predicted_token = predict_next_token(prev_token)
print(f"The predicted token after '{prev_token}' is '{predicted_token}'.")

The predicted token after 'Qed' is '.'.


In [ ]:
data_test = df_test.apply(lambda x: tokenizer(x['combined']), axis=1).tolist()
